In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import parse_json, flatten, col,listagg

session = get_active_session()


In [ ]:
df_movies = session.table("MOVIES_METADATA").filter(col("OVERVIEW").is_not_null());
print(df_movies.count())
df_movies.show()

In [ ]:
df_movies.select(col("ORIGINAL_LANGUAGE")).distinct().limit(10)

In [ ]:
df_movies.columns

In [ ]:
df_genres_parsed = df_movies.select(col("ID"),flatten(parse_json(col("GENRES")))).select(
    col("ID"),col("VALUE")["name"].alias("genre_name")
).group_by("ID").agg(
    listagg(col("genre_name"), ", ").as_("genre_list"))

df_final = df_movies.join(df_genres_parsed, "ID").select(
    df_movies["*"],  # Select all columns from the initial DataFrame
    col("genre_list")  # Add the aggregated genre list
)
df_final.show()

In [ ]:
# Calculate weighted rate (IMDb formula)
# weighted rating (WR) = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C where: R = average for the movie (mean) = (Rating) v = number of votes for the movie = (votes) m = minimum votes required to be listed in the Top 250 (currently 25000)
df_final = df_final.with_column('WEIGHTED_RATE', df_final["VOTE_COUNT"] / (df_final["VOTE_COUNT"] + 10 ) * df_final["VOTE_AVERAGE"] + (10 / (df_final["VOTE_COUNT"] + 10)) * 5.0) 

In [ ]:
df_final.show()

In [ ]:
df_final.order_by(col("WEIGHTED_RATE").desc()).select(col("TITLE"), col("GENRE_LIST"), col("TAGLINE"), col("IMDB_ID"), col("ORIGINAL_LANGUAGE"), col("VOTE_AVERAGE")).limit(10)


In [ ]:
df_movies_set = df_final[["TITLE","GENRE_LIST", "IMDB_ID", "OVERVIEW", "WEIGHTED_RATE", "ORIGINAL_LANGUAGE"]]
df_movies_set.show()

In [ ]:
df_movies_set.write.mode("overwrite").save_as_table("MOVIES_PROCESSED")

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE movie_recommender
  ON OVERVIEW
  ATTRIBUTES ORIGINAL_LANGUAGE
  WAREHOUSE = COMPUTE_WH
  TARGET_LAG = '1 day'
  AS (
    SELECT
        TITLE,
        OVERVIEW,
        IMDB_ID,
        ORIGINAL_LANGUAGE,
        WEIGHTED_RATE
    FROM MOVIES_PROCESSED
);

In [ ]:
from snowflake.core import Root

root = Root(session)

movie_recommender_service = (root
  .databases["MOVIES_DB"]
  .schemas["PUBLIC"]
  .cortex_search_services["movie_recommender"]
)

resp = movie_recommender_service.search(
  query="I realy liked Being John Malkovich What other movies should I watch?",
  columns=["TITLE", "OVERVIEW", "IMDB_ID"],
  filter={"@eq": {"ORIGINAL_LANGUAGE": "en"} },
  limit=3
)
print(resp.to_json())

In [ ]:
resp = movie_recommender_service.search(
  query="I realy liked Pulp Fiction. What other movies should I watch.",
  columns=["TITLE"],
  filter={"@eq": {"ORIGINAL_LANGUAGE": "ro"} },
  limit=3
)
print(resp.to_json())